## Q 2a) Fine-tune the pre-trained BERT model for the UPOS prediction task.


In [52]:
pip install conllu


Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [54]:
pip install seqeval

Note: you may need to restart the kernel to use updated packages.


In [55]:
import conllu
from conllu import parse

data_file = open("hi_hdtb-ud-dev.conllu", "r", encoding="utf-8").read()
sentences = parse(data_file)
len(sentences)

1659

In [56]:
sentences = [sentence.serialize() for sentence in sentences]

In [57]:
from sklearn.model_selection import train_test_split

## Split size may be adjusted here
train, test = train_test_split(sentences,test_size=0.20)
train, val = train_test_split(train,test_size=0.10)

In [58]:
len(train),len(val),len(test)

(1194, 133, 332)

In [59]:
import os

## Create directory data/ if does not exist
if not os.path.exists('data'):
    os.makedirs('data')
    
for l,s in zip([train,val,test],['train','validation','test']):
    with open(f'data/{s}.conllu','w',encoding='utf-8') as fp:
        fp.write(''.join(l))

In [60]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [61]:
from datasets import load_dataset

In [62]:
dataset = load_dataset('ConlluIndic.py')
dataset

Found cached dataset conllu_indic (C:/Users/abhil/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/50260ca812993370c93c5949d08ad1fb388e0fced25f584d6ede7b09423bc32c)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 1194
    })
    validation: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 133
    })
    test: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 332
    })
})

In [63]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [64]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [65]:
from transformers import AutoTokenizer

model_checkpoint = 'ai4bharat/indic-bert'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\abhil/.cache\huggingface\hub\models--ai4bharat--indic-bert\snapshots\4842dd258ecc0546f0d660b76a3b22a9c632f401\config.json
Model config AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24

In [66]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            #if label % 2 == 1:
            #    label += 1
            new_labels.append(label)

    return new_labels


In [67]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["upos_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [68]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


Loading cached processed dataset at C:/Users/abhil/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/50260ca812993370c93c5949d08ad1fb388e0fced25f584d6ede7b09423bc32c\cache-fa75edc484da9b39.arrow
Loading cached processed dataset at C:/Users/abhil/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/50260ca812993370c93c5949d08ad1fb388e0fced25f584d6ede7b09423bc32c\cache-780f601eaf1c85b7.arrow
Loading cached processed dataset at C:/Users/abhil/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/50260ca812993370c93c5949d08ad1fb388e0fced25f584d6ede7b09423bc32c\cache-016a86a47ba84b80.arrow


In [69]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric, load_from_disk, load_dataset
import numpy as np

In [70]:
model_checkpoint = 'ai4bharat/indic-bert'


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = load_from_disk(r'C:\Users\abhil\tokenized_datasets\tokenized_datasets')



data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = load_metric('seqeval')



dataset = load_dataset('ConlluIndic.py')

upos_feature = dataset['train'].features['upos_tags']
label_names = upos_feature.feature.names



id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}



model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label,label2id=label2id)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\abhil/.cache\huggingface\hub\models--ai4bharat--indic-bert\snapshots\4842dd258ecc0546f0d660b76a3b22a9c632f401\config.json
Model config AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24

  0%|          | 0/3 [00:00<?, ?it/s]

loading configuration file config.json from cache at C:\Users\abhil/.cache\huggingface\hub\models--ai4bharat--indic-bert\snapshots\4842dd258ecc0546f0d660b76a3b22a9c632f401\config.json
Model config AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "id2label": {
    "0": "ADJ",
    "1": "ADP",
    "10": "PRON",
    "11": "PROPN",
    "12": "PUNCT",
    "13": "SCONJ",
    "14": "SYM",
    "15": "VERB",
    "16": "X",
    "17": "_",
    "2": "ADV",
    "3": "AUX",
    "4": "CCONJ",
    "5": "DET",
    "6": "INTJ",
    "7": "NOUN",
    "8": "NUM",
    "9": "PART"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "label2id": {
    "ADJ": "0",
    "ADP": "1",
    "ADV": "2",
    "AUX": "3",
   

In [71]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


In [85]:
batch_size = 10
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy="epoch",
    learning_rate=2e-4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    #fp16= True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    #eval_dataset=tokenized_datasets["validation"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [79]:
trainer.train()


C:\Users\abhil\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1194
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 1200
  Number of trainable parameters = 32866834


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.608646,0.860224,0.861296,0.860759,0.892724
2,No log,0.587595,0.840503,0.860050,0.850164,0.882463
3,No log,0.507979,0.864809,0.863372,0.864090,0.892491
4,No log,0.566897,0.867188,0.875831,0.871488,0.897854
5,0.047300,0.583324,0.869979,0.872508,0.871242,0.902519
6,0.047300,0.599047,0.880972,0.888289,0.884615,0.905317
7,0.047300,0.609163,0.886092,0.891611,0.888843,0.907882
8,0.047300,0.575716,0.887289,0.895764,0.891507,0.913246
9,0.008500,0.584627,0.887747,0.896595,0.892149,0.913246
10,0.008500,0.590251,0.888158,0.897010,0.892562,0.913246


***** Running Evaluation *****
  Num examples = 133
  Batch size = 10
Saving model checkpoint to model/upos\checkpoint-120
Configuration saved in model/upos\checkpoint-120\config.json
Model weights saved in model/upos\checkpoint-120\pytorch_model.bin
tokenizer config file saved in model/upos\checkpoint-120\tokenizer_config.json
Special tokens file saved in model/upos\checkpoint-120\special_tokens_map.json
Deleting older checkpoint [model\upos\checkpoint-1200] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 133
  Batch size = 10
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\seque

Saving model checkpoint to model/upos\checkpoint-480
Configuration saved in model/upos\checkpoint-480\config.json
Model weights saved in model/upos\checkpoint-480\pytorch_model.bin
tokenizer config file saved in model/upos\checkpoint-480\tokenizer_config.json
Special tokens file saved in model/upos\checkpoint-480\special_tokens_map.json
Deleting older checkpoint [model\upos\checkpoint-360] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 133
  Batch size = 10
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PART seems not to be NE tag.
  warnin

Saving model checkpoint to model/upos\checkpoint-840
Configuration saved in model/upos\checkpoint-840\config.json
Model weights saved in model/upos\checkpoint-840\pytorch_model.bin
tokenizer config file saved in model/upos\checkpoint-840\tokenizer_config.json
Special tokens file saved in model/upos\checkpoint-840\special_tokens_map.json
Deleting older checkpoint [model\upos\checkpoint-720] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 133
  Batch size = 10
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PART seems not to be NE tag.
  warnin

Saving model checkpoint to model/upos\checkpoint-1200
Configuration saved in model/upos\checkpoint-1200\config.json
Model weights saved in model/upos\checkpoint-1200\pytorch_model.bin
tokenizer config file saved in model/upos\checkpoint-1200\tokenizer_config.json
Special tokens file saved in model/upos\checkpoint-1200\special_tokens_map.json
Deleting older checkpoint [model\upos\checkpoint-1080] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1200, training_loss=0.02337848622351885, metrics={'train_runtime': 3397.9791, 'train_samples_per_second': 3.514, 'train_steps_per_second': 0.353, 'total_flos': 32022918557328.0, 'train_loss': 0.02337848622351885, 'epoch': 10.0})

In [86]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 133
  Batch size = 10


{'eval_loss': 0.5902513861656189,
 'eval_precision': 0.8881578947368421,
 'eval_recall': 0.8970099667774086,
 'eval_f1': 0.8925619834710744,
 'eval_accuracy': 0.9132462686567164,
 'eval_runtime': 13.3108,
 'eval_samples_per_second': 9.992,
 'eval_steps_per_second': 1.052}

In [82]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 332
  Batch size = 10


C:\Users\abhil\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.627139687538147,
 'eval_precision': 0.8841870824053452,
 'eval_recall': 0.898771021992238,
 'eval_f1': 0.8914194065757818,
 'eval_accuracy': 0.9079005184989893,
 'eval_runtime': 27.526,
 'eval_samples_per_second': 12.061,
 'eval_steps_per_second': 1.235}

## 2)b) The micro precison is 0.8881578947368421.
##         The micro recall is 0.8970099667774086. 
##         The micro F1 score is 0.8925619834710744.
##         The macro Precision is 0.886172488565.
##         The macro recall is 0.89789049438.
##          The macro F1 score is 0.891990695023.

